In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import plot_model
import pandas as pd
import math
import warnings

import tweepy

In [ ]:
df = pd.read_excel("C://Users\guysh\OneDrive\Documents\Guy School\הנדסת תוכנה\יב\פרויקט סופי\senators and their ages.xlsx")

In [ ]:
df = df.drop(['State Name','state abv','Abbrev','First Name','Last Name'], axis=1)

df

In [ ]:
api_key = "aAscR03t3Q3t93Ved1Gia1Uak"
api_key_secret = "ZAto2kY6cQDIRc4Ez0LJSlU7TFgpkq9IsVpidZrAE8IGmlAsgZ"
api_bearer_token = "AAAAAAAAAAAAAAAAAAAAACtmhAEAAAAAywHX4WJjRAxM4s9TmYPa8Uu5Az8%3D7aLtKEaihvIFxqrTDhV5xXwoXstfrWx5Dimr0bLrghi8E2ZYjG"
access_token = "1042055001554780161-JIGrhoveobzmKo5NekGVuaKq293dtM"
access_token_secret = "Ss7mMzXvZhzwe0x3aC0SjRQI9ILTEEW3BonvL9k4zS6iY"

auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [ ]:
def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #initialize a list to hold all the tweepy Tweets
    all_tweets = []  
    
    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    all_tweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = all_tweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:        
        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)
        
        #save most recent tweets
        all_tweets.extend(new_tweets)
        
        #update the id of the oldest tweet less one
        oldest = all_tweets[-1].id - 1
        
    print(f"...{len(all_tweets)} tweets downloaded")
    return all_tweets

In [ ]:
# create a function which gets a list of tweets information about each senator and adds it as features of each of their tweets
def add_feats_to_tweet(df):
    tweets_df = pd.DataFrame(columns=['handle', 'tweet_text', 'tweet_time','state abv','state name','senator name','party','age'])

    for index, row in df.iterrows():
        # Get the Twitter handle for the current row
        handle = row['Twitter Handle']

        # Use the tweepy API to get all tweets from the account
        print(handle)

        tweets = get_all_tweets(handle)

        # Loop through the tweets and add them to the DataFrame
        for tweet in tweets:
            tweets_df = tweets_df.append({
                'handle': row['Twitter Handle'],
                'tweet_text': tweet.text,
                'tweet_time': tweet.created_at,
                'state abv':row['State Abv'],
                'state name':row['State'],
                'senator name':row['Name'],
                'party':row['Party'],
                'age':row['Age']
            }, ignore_index=True)
    return tweets_df

In [ ]:
warnings.filterwarnings('ignore')
tweets_df = add_feats_to_tweet(df)
warnings.filterwarnings('default')

In [ ]:
tweets_df

In [ ]:
type(tweets_df)

In [ ]:
tweets_df['tweet_time'] = tweets_df['tweet_time'].apply(lambda x: x.tz_localize(None))

In [ ]:
tweets_df.to_excel("C://Users\guysh\OneDrive\Documents\Guy School\הנדסת תוכנה\יב\פרויקט סופי\ tweets dataset.xlsx", index=False)

In [ ]:
tweets_df.to_csv("C://Users\guysh\OneDrive\Documents\Guy School\הנדסת תוכנה\יב\פרויקט סופי\ tweets dataset csv", index=False)